In [3]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import os
import random

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
)
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

from torch.utils.tensorboard import SummaryWriter

In [5]:
# specify which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [6]:
# reproducibility
RANDOM_STATE = 0
torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)


## Package Loading

In [7]:
DIR = os.getcwd()
if DIR.endswith("code"):
    DIR = DIR[:-5]

from utils.plot_util import plot_one_gt_train_test, plot_gt_pre_overlap, plot_3_losses
from utils.data_util import (
    dotdict,
    load_pos_s21,
    save_pre,
    load_pre,
    PreprocessedDataset,
    get_dataloader,
)
from utils.nn_util import dict_act, NN, NN_fe, count_parameters,count_parameters_table, CNN_baseline, CNN_baseline_fe
from utils.complex_util import dict_act_complex, complex_normalize, NN_complex,NN_complex_concat, NN_complex_fe, NN_complex_fe_concat, CNN_baseline_complex, CNN_baseline_complex_fe


# real valued NN

## Data Extraction

In [8]:
# load data
folder = os.path.join(DIR, "data")
data_file_name = os.path.join(folder, "data_offset.pkl")
pos, s21, pos_TEST, s21_TEST = load_pos_s21(data_file_name, 5)

X = s21
Y = pos

X_TEST = s21_TEST
y_TEST = pos_TEST


## Data Preprocessing

### Train test split wit indicies

In [9]:
indices = np.arange(X.shape[0])
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, Y, indices, test_size=0.2, random_state=RANDOM_STATE
)
idx_train_sort = np.sort(idx_train)
idx_test_sort = np.sort(idx_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(411257, 242)
(411257,)
(102815, 242)
(102815,)


### Normalization

In [10]:
scaler = StandardScaler()
scaler.fit(X_train)
X_scaled = scaler.transform(X)
X_train_scaled = X_scaled[idx_train]
X_test_scaled = X_scaled[idx_test]
X_TEST_scaled = scaler.transform(X_TEST)

y_train = Y[idx_train].reshape(-1, 1)  # reshape to (N,1)
y_test = Y[idx_test].reshape(-1, 1)  # reshape to (N,1)
y_TEST = y_TEST.reshape(-1, 1)  # reshape to (N,1)

print(X_train_scaled.shape)
print(y_train.shape)
print(X_test_scaled.shape)
print(y_test.shape)
print(X_TEST_scaled.shape)
print(y_TEST.shape)


(411257, 242)
(411257, 1)
(102815, 242)
(102815, 1)
(20135, 242)
(20135, 1)


## Model Comparison

### NN baseline

In [11]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/NN_tuned_nonorm")
MODEL_NAME = "NN_32-16_sigmoid.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 242,
 'list_hidden': [32, 16],
 'activation': 'sigmoid',
 'batch_size': 128,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': False}

In [16]:
# dot access
config = dotdict(_config)

model = NN(
    n_features=config.n_features,
    list_hidden=config.list_hidden,
    activation_fn=dict_act[config.activation],
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

NN(
  (activation): Sigmoid()
  (layers): Sequential(
    (0): Linear(in_features=242, out_features=32, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [17]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.Tensor(X_scaled).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.Tensor(X_TEST_scaled).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 1.99 mm
test RMSE: 2.12 mm
TEST RMSE: 5.66 mm
TEST MAE: 2.04 mm
TEST ME: -0.12 mm


### NN bn

In [ ]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/NN_tuned")
MODEL_NAME = "NN_32-16_sigmoid.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 242,
 'list_hidden': [32, 16],
 'activation': 'sigmoid',
 'batch_size': 128,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': True}

In [ ]:
# dot access
config = dotdict(_config)

model = NN(
    n_features=config.n_features,
    list_hidden=config.list_hidden,
    activation_fn=dict_act[config.activation],
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

NN(
  (activation): Sigmoid()
  (layers): Sequential(
    (0): Linear(in_features=242, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Sigmoid()
    (6): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [ ]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.Tensor(X_scaled).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.Tensor(X_TEST_scaled).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 2.71 mm
test RMSE: 2.73 mm
TEST RMSE: 3.89 mm
TEST MAE: 2.08 mm
TEST ME: -0.18 mm


### NN fe

In [28]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/NN_tuned_fe")
MODEL_NAME = "NN_32-16.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 242,
 'list_hidden': [32, 16],
 'activation': 'sigmoid',
 'batch_size': 128,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': False}

In [30]:
# dot access
config = dotdict(_config)

model = NN_fe(
    n_features=config.n_features,
    list_hidden=config.list_hidden,
    activation_fn=dict_act[config.activation],
    device=device,
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
    ues_ln=config.ues_ln,
)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

NN_fe(
  (activation): Sigmoid()
  (layers): Sequential(
    (0): Linear(in_features=242, out_features=32, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [31]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.Tensor(X_scaled).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.Tensor(X_TEST_scaled).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 1.17 mm
test RMSE: 1.21 mm
TEST RMSE: 2.50 mm
TEST MAE: 1.24 mm
TEST ME: -0.11 mm


### CNN baseline

In [11]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/CNN")
MODEL_NAME = "cnn_baseline.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'base_channels': 8,
 'activation': 'sigmoid',
 'batch_size': 64,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': False}

In [13]:
# dot access
config = dotdict(_config)

model = CNN_baseline(
    base_channels=config.base_channels,
    activation_fn=dict_act[config.activation],
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
).to(device)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

CNN_baseline(
  (conv1): Conv1d(1, 8, kernel_size=(22,), stride=(11,))
  (conv2): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
  (conv3): Conv1d(16, 32, kernel_size=(3,), stride=(2,))
  (conv4): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (conv_block1): Sequential(
    (0): Conv1d(1, 8, kernel_size=(22,), stride=(11,))
    (1): Sigmoid()
  )
  (conv_block2): Sequential(
    (0): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
    (1): Sigmoid()
  )
  (conv_block3): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(2,))
    (1): Sigmoid()
  )
  (conv_block4): Sequential(
    (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
    (1): Sigmoid()
  )
)

In [15]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.Tensor(X_scaled).unsqueeze(1).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.Tensor(X_TEST_scaled).unsqueeze(1).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 0.73 mm
test RMSE: 0.75 mm
TEST RMSE: 1.74 mm
TEST MAE: 1.03 mm
TEST ME: 0.11 mm


### CNN_bn

In [16]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/CNN")
MODEL_NAME = "cnn_baseline_bn.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'base_channels': 8,
 'activation': 'sigmoid',
 'batch_size': 64,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': True}

In [17]:
# dot access
config = dotdict(_config)

model = CNN_baseline(
    base_channels=config.base_channels,
    activation_fn=dict_act[config.activation],
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
).to(device)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

CNN_baseline(
  (conv1): Conv1d(1, 8, kernel_size=(22,), stride=(11,))
  (conv2): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
  (conv3): Conv1d(16, 32, kernel_size=(3,), stride=(2,))
  (conv4): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (conv_block1): Sequential(
    (0): Conv1d(1, 8, kernel_size=(22,), stride=(11,))
    (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
  )
  (conv_block2): Sequential(
    (0): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
  )
  (conv_block3): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(2,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
  )
  (conv_block4): Sequential(
    (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
    (1): BatchNorm1d(64, eps=1

In [18]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.Tensor(X_scaled).unsqueeze(1).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.Tensor(X_TEST_scaled).unsqueeze(1).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 1.84 mm
test RMSE: 1.86 mm
TEST RMSE: 2.67 mm
TEST MAE: 1.89 mm
TEST ME: -0.08 mm


### CNN fe

In [44]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/CNN_fe")
MODEL_NAME = "cnn_baseline.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 242,
 'base_channels': 8,
 'activation': 'sigmoid',
 'batch_size': 64,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'betas': (0.9, 0.999),
 'use_fe': True,
 'use_dropout': False,
 'use_bn': False}

In [45]:
# dot access
config = dotdict(_config)

model = CNN_baseline_fe(
    n_features=config.n_features,
    base_channels=config.base_channels,
    device=device,
    activation_fn=dict_act[config.activation],
    use_fe=config.use_fe,
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
).to(device)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

CNN_baseline_fe(
  (conv1): Conv1d(1, 8, kernel_size=(22,), stride=(11,))
  (conv2): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
  (conv3): Conv1d(16, 32, kernel_size=(3,), stride=(2,))
  (conv4): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (conv_block1): Sequential(
    (0): Conv1d(1, 8, kernel_size=(22,), stride=(11,))
    (1): Sigmoid()
  )
  (conv_block2): Sequential(
    (0): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
    (1): Sigmoid()
  )
  (conv_block3): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(2,))
    (1): Sigmoid()
  )
  (conv_block4): Sequential(
    (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
    (1): Sigmoid()
  )
)

In [46]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.Tensor(X_scaled).unsqueeze(1).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.Tensor(X_TEST_scaled).unsqueeze(1).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 0.56 mm
test RMSE: 0.57 mm
TEST RMSE: 1.47 mm
TEST MAE: 0.80 mm
TEST ME: 0.11 mm


## Activation Comparison

#### sigmoid

In [ ]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/NN_tuned")
MODEL_NAME = "NN_32-16_sigmoid.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 242,
 'list_hidden': [32, 16],
 'activation': 'sigmoid',
 'batch_size': 128,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': True}

In [ ]:
# dot access
config = dotdict(_config)

model = NN(
    n_features=config.n_features,
    list_hidden=config.list_hidden,
    activation_fn=dict_act[config.activation],
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

NN(
  (activation): Sigmoid()
  (layers): Sequential(
    (0): Linear(in_features=242, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Sigmoid()
    (6): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [ ]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.Tensor(X_scaled).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.Tensor(X_TEST_scaled).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 2.71 mm
test RMSE: 2.73 mm
TEST RMSE: 3.89 mm
TEST MAE: 2.08 mm
TEST ME: -0.18 mm


#### tanh

In [35]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/NN_tuned")
MODEL_NAME = "NN_32-16.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 242,
 'list_hidden': [32, 16],
 'activation': 'tanh',
 'batch_size': 128,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': True}

In [36]:
# dot access
config = dotdict(_config)

model = NN(
    n_features=config.n_features,
    list_hidden=config.list_hidden,
    activation_fn=dict_act[config.activation],
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

NN(
  (activation): Tanh()
  (layers): Sequential(
    (0): Linear(in_features=242, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Tanh()
    (6): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [37]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.Tensor(X_scaled).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.Tensor(X_TEST_scaled).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 4.64 mm
test RMSE: 4.66 mm
TEST RMSE: 6.20 mm
TEST MAE: 3.77 mm
TEST ME: -0.14 mm


#### selu

In [38]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/NN_tuned")
MODEL_NAME = "NN_32-16_selu.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 242,
 'list_hidden': [32, 16],
 'activation': 'selu',
 'batch_size': 128,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': True}

In [39]:
# dot access
config = dotdict(_config)

model = NN(
    n_features=config.n_features,
    list_hidden=config.list_hidden,
    activation_fn=dict_act[config.activation],
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

NN(
  (activation): SELU()
  (layers): Sequential(
    (0): Linear(in_features=242, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SELU()
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): SELU()
    (6): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [40]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.Tensor(X_scaled).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.Tensor(X_TEST_scaled).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 4.84 mm
test RMSE: 4.86 mm
TEST RMSE: 5.88 mm
TEST MAE: 4.30 mm
TEST ME: -0.08 mm


#### relu

In [41]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/NN_tuned")
MODEL_NAME = "NN_32-16_relu.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 242,
 'list_hidden': [32, 16],
 'activation': 'relu',
 'batch_size': 128,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': True}

In [42]:
# dot access
config = dotdict(_config)

model = NN(
    n_features=config.n_features,
    list_hidden=config.list_hidden,
    activation_fn=dict_act[config.activation],
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

NN(
  (activation): ReLU()
  (layers): Sequential(
    (0): Linear(in_features=242, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [43]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.Tensor(X_scaled).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.Tensor(X_TEST_scaled).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 4.26 mm
test RMSE: 4.30 mm
TEST RMSE: 5.96 mm
TEST MAE: 4.00 mm
TEST ME: -0.06 mm


#### leakyrelu

In [44]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/NN_tuned")
MODEL_NAME = "NN_32-16_leakyrelu.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 242,
 'list_hidden': [32, 16],
 'activation': 'tanh',
 'batch_size': 128,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': True}

In [45]:
# dot access
config = dotdict(_config)

model = NN(
    n_features=config.n_features,
    list_hidden=config.list_hidden,
    activation_fn=dict_act[config.activation],
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

NN(
  (activation): Tanh()
  (layers): Sequential(
    (0): Linear(in_features=242, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Tanh()
    (6): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [46]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.Tensor(X_scaled).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.Tensor(X_TEST_scaled).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 5.58 mm
test RMSE: 5.60 mm
TEST RMSE: 8.30 mm
TEST MAE: 4.79 mm
TEST ME: 0.00 mm


# CVNN

## Data Extraction

In [11]:
# load data
folder = os.path.join(DIR, "data")
data_file_name = os.path.join(folder, "data_offset_complex.pkl")
pos, s21, pos_TEST, s21_TEST = load_pos_s21(data_file_name, 5)

X = s21
Y = pos

X_TEST = s21_TEST
y_TEST = pos_TEST


## Data Preprocessing

### Train test split wit indicies

In [12]:
indices = np.arange(X.shape[0])
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, Y, indices, test_size=0.2, random_state=RANDOM_STATE
)
idx_train_sort = np.sort(idx_train)
idx_test_sort = np.sort(idx_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(411257, 121)
(411257,)
(102815, 121)
(102815,)


### Normalization

In [13]:
X_train_scaled, X_train_mean, X_train_std = complex_normalize(X_train)
X_test_scaled, X_test_mean, X_test_std = complex_normalize(X_test, X_train_mean, X_train_std)
X_TEST_scaled, X_TEST_mean, X_TEST_std = complex_normalize(X_TEST, X_train_mean, X_train_std)

X_scaled = np.zeros_like(X)
X_scaled[idx_train] = X_train_scaled
X_scaled[idx_test] = X_test_scaled

y_train = Y[idx_train].reshape(-1, 1)  # reshape to (N,1)
y_test = Y[idx_test].reshape(-1, 1)  # reshape to (N,1)
y_TEST = y_TEST.reshape(-1, 1)  # reshape to (N,1)

print(X_train_scaled.shape)
print(y_train.shape)
print(X_test_scaled.shape)
print(y_test.shape)
print(X_TEST_scaled.shape)
print(y_TEST.shape)


(411257, 121)
(411257, 1)
(102815, 121)
(102815, 1)
(20135, 121)
(20135, 1)


## Model Comparison

### CVNN

In [17]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/NN_tuned_complex_concat")
MODEL_NAME = "NN_32-16.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 121,
 'list_hidden': [32, 16],
 'activation': 'sigmoid',
 'batch_size': 128,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0004,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': False}

In [19]:
# dot access
config = dotdict(_config)

model = NN_complex_concat(
    n_features=config.n_features,
    list_hidden=config.list_hidden,
    activation_fn=dict_act_complex[config.activation],
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

NN_complex_concat(
  (activation): ComplexSigmoid()
  (layers): Sequential(
    (0): ComplexLinear(
      (fc_r): Linear(in_features=121, out_features=32, bias=True)
      (fc_i): Linear(in_features=121, out_features=32, bias=True)
    )
    (1): ComplexSigmoid()
    (2): ComplexLinear(
      (fc_r): Linear(in_features=32, out_features=16, bias=True)
      (fc_i): Linear(in_features=32, out_features=16, bias=True)
    )
    (3): ComplexSigmoid()
  )
  (out): Linear(in_features=32, out_features=1, bias=True)
)

In [20]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.from_numpy(X_scaled).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.from_numpy(X_TEST_scaled).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 0.74 mm
test RMSE: 0.75 mm
TEST RMSE: 1.53 mm
TEST MAE: 0.95 mm
TEST ME: -0.07 mm


### CVNN fe

#### concat

In [25]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/NN_tuned_complex_fe_concat")
MODEL_NAME = "NN_32-16.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 121,
 'list_hidden': [32, 16],
 'activation': 'sigmoid',
 'batch_size': 64,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': False}

In [27]:
# dot access
config = dotdict(_config)

model = NN_complex_fe_concat(
    n_features=config.n_features,
    list_hidden=config.list_hidden,
    activation_fn=dict_act_complex[config.activation],
    device=device,
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

NN_complex_fe_concat(
  (activation): ComplexSigmoid()
  (layers): Sequential(
    (0): ComplexLinear(
      (fc_r): Linear(in_features=121, out_features=32, bias=True)
      (fc_i): Linear(in_features=121, out_features=32, bias=True)
    )
    (1): ComplexSigmoid()
    (2): ComplexLinear(
      (fc_r): Linear(in_features=32, out_features=16, bias=True)
      (fc_i): Linear(in_features=32, out_features=16, bias=True)
    )
    (3): ComplexSigmoid()
  )
  (out): Linear(in_features=32, out_features=1, bias=True)
)

In [28]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.from_numpy(X_scaled).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.from_numpy(X_TEST_scaled).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 0.87 mm
test RMSE: 0.89 mm
TEST RMSE: 1.58 mm
TEST MAE: 0.93 mm
TEST ME: -0.01 mm


#### abs

In [11]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/NN_tuned_complex_fe")
MODEL_NAME = "NN_32-16.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 121,
 'list_hidden': [32, 16],
 'activation': 'sigmoid',
 'batch_size': 128,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': False}

In [13]:
# dot access
config = dotdict(_config)

model = NN_complex_fe(
    n_features=config.n_features,
    list_hidden=config.list_hidden,
    activation_fn=dict_act_complex[config.activation],
    device=device,
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

NN_complex_fe(
  (activation): ComplexSigmoid()
  (layers): Sequential(
    (0): ComplexLinear(
      (fc_r): Linear(in_features=121, out_features=32, bias=True)
      (fc_i): Linear(in_features=121, out_features=32, bias=True)
    )
    (1): ComplexSigmoid()
    (2): ComplexLinear(
      (fc_r): Linear(in_features=32, out_features=16, bias=True)
      (fc_i): Linear(in_features=32, out_features=16, bias=True)
    )
    (3): ComplexSigmoid()
    (4): ComplexLinear(
      (fc_r): Linear(in_features=16, out_features=1, bias=True)
      (fc_i): Linear(in_features=16, out_features=1, bias=True)
    )
  )
)

In [14]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.from_numpy(X_scaled).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.from_numpy(X_TEST_scaled).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 0.69 mm
test RMSE: 0.71 mm
TEST RMSE: 1.23 mm
TEST MAE: 0.75 mm
TEST ME: -0.07 mm


### CVCNN

In [21]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/CNN_complex")
MODEL_NAME = "cnn_baseline.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'base_channels': 4,
 'activation': 'sigmoid',
 'batch_size': 64,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.001,
 'betas': (0.9, 0.999),
 'use_dropout': False,
 'use_bn': False}

In [22]:
# dot access
config = dotdict(_config)

model = CNN_baseline_complex(
    base_channels=config.base_channels,
    activation_fn=dict_act_complex[config.activation],
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
).to(device)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

CNN_baseline_complex(
  (conv1): ComplexConv1d(
    (conv_r): Conv1d(1, 4, kernel_size=(11,), stride=(5,))
    (conv_i): Conv1d(1, 4, kernel_size=(11,), stride=(5,))
  )
  (conv2): ComplexConv1d(
    (conv_r): Conv1d(4, 8, kernel_size=(3,), stride=(2,))
    (conv_i): Conv1d(4, 8, kernel_size=(3,), stride=(2,))
  )
  (conv3): ComplexConv1d(
    (conv_r): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
    (conv_i): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
  )
  (conv4): ComplexConv1d(
    (conv_r): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (conv_i): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
  )
  (conv5): ComplexConv1d(
    (conv_r): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
    (conv_i): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  )
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (conv_block1): Sequential(
    (0): ComplexConv1d(
      (conv_r): Conv1d(1, 4, kernel_size=(11,), stride=(5,))
      (conv_i): Conv1d(1, 4, kernel_size=(11,), stride=(5,))
   

In [23]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.from_numpy(X_scaled).unsqueeze(1).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.from_numpy(X_TEST_scaled).unsqueeze(1).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 1.00 mm
test RMSE: 1.00 mm
TEST RMSE: 1.53 mm
TEST MAE: 1.02 mm
TEST ME: 0.45 mm


### CVCNN fe

In [24]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/CNN_complex_fe")
MODEL_NAME = "cnn_baseline.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

_config

{'n_features': 121,
 'base_channels': 4,
 'activation': 'sigmoid',
 'batch_size': 64,
 'epochs': 500,
 'decay_after': 100,
 'patience': 16,
 'lr': 0.001,
 'weight_decay': 0.0001,
 'betas': (0.9, 0.999),
 'use_fe': True,
 'use_dropout': False,
 'use_bn': False}

In [25]:
# dot access
config = dotdict(_config)

model = CNN_baseline_complex_fe(
    n_features=config.n_features,
    base_channels=config.base_channels,
    device=device,
    activation_fn=dict_act_complex[config.activation],
    use_fe=config.use_fe,
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
).to(device)

model.load_state_dict(torch.load(os.path.join(SAVED_IN, MODEL_NAME), map_location=device))
model.to(device)
model.eval()

CNN_baseline_complex_fe(
  (conv1): ComplexConv1d(
    (conv_r): Conv1d(1, 4, kernel_size=(11,), stride=(5,))
    (conv_i): Conv1d(1, 4, kernel_size=(11,), stride=(5,))
  )
  (conv2): ComplexConv1d(
    (conv_r): Conv1d(4, 8, kernel_size=(3,), stride=(2,))
    (conv_i): Conv1d(4, 8, kernel_size=(3,), stride=(2,))
  )
  (conv3): ComplexConv1d(
    (conv_r): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
    (conv_i): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
  )
  (conv4): ComplexConv1d(
    (conv_r): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (conv_i): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
  )
  (conv5): ComplexConv1d(
    (conv_r): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
    (conv_i): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  )
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (conv_block1): Sequential(
    (0): ComplexConv1d(
      (conv_r): Conv1d(1, 4, kernel_size=(11,), stride=(5,))
      (conv_i): Conv1d(1, 4, kernel_size=(11,), stride=(5,))


In [26]:
# evaluation on  training, test and TEST set

with torch.no_grad():
    y_ = model(torch.from_numpy(X_scaled).unsqueeze(1).to(device)).cpu().numpy().squeeze()
    y_TEST_ = model(torch.from_numpy(X_TEST_scaled).unsqueeze(1).to(device)).cpu().numpy().squeeze()
y_train_ = y_[idx_train]
y_test_ = y_[idx_test]

RMSE_test_final_mm = np.sqrt(mean_squared_error(y_test, y_test_))
RMSE_train_final_mm = np.sqrt(mean_squared_error(y_train, y_train_))
RMSE_TEST_final_mm = np.sqrt(mean_squared_error(y_TEST, y_TEST_))
MAE_TEST_final_mm = mean_absolute_error(y_TEST, y_TEST_)
ME_TEST_final_mm = np.mean(y_TEST - y_TEST_)

print(f"train RMSE: {RMSE_train_final_mm:.2f} mm")
print(f"test RMSE: {RMSE_test_final_mm:.2f} mm")
print(f"TEST RMSE: {RMSE_TEST_final_mm:.2f} mm")
print(f"TEST MAE: {MAE_TEST_final_mm:.2f} mm")
print(f"TEST ME: {ME_TEST_final_mm:.2f} mm")

train RMSE: 0.73 mm
test RMSE: 0.73 mm
TEST RMSE: 1.01 mm
TEST MAE: 0.74 mm
TEST ME: 0.18 mm


## Model parameters
from nn_util import count_parameters_table


### CVCNN_FE

In [20]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/CNN_complex_fe")
MODEL_NAME = "cnn_baseline.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

In [21]:
# dot access
config = dotdict(_config)

cvcnn = CNN_baseline_complex_fe(
    n_features=config.n_features,
    base_channels=config.base_channels,
    device=device,
    activation_fn=dict_act_complex[config.activation],
    use_fe=config.use_fe,
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
).to(device)

CNN_baseline_complex_fe(
  (conv1): ComplexConv1d(
    (conv_r): Conv1d(1, 4, kernel_size=(11,), stride=(5,))
    (conv_i): Conv1d(1, 4, kernel_size=(11,), stride=(5,))
  )
  (conv2): ComplexConv1d(
    (conv_r): Conv1d(4, 8, kernel_size=(3,), stride=(2,))
    (conv_i): Conv1d(4, 8, kernel_size=(3,), stride=(2,))
  )
  (conv3): ComplexConv1d(
    (conv_r): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
    (conv_i): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
  )
  (conv4): ComplexConv1d(
    (conv_r): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (conv_i): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
  )
  (conv5): ComplexConv1d(
    (conv_r): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
    (conv_i): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  )
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (conv_block1): Sequential(
    (0): ComplexConv1d(
      (conv_r): Conv1d(1, 4, kernel_size=(11,), stride=(5,))
      (conv_i): Conv1d(1, 4, kernel_size=(11,), stride=(5,))


In [22]:
count_parameters_table(cvcnn)

+---------------------+------------+
|       Modules       | Parameters |
+---------------------+------------+
|        weight       |    121     |
| conv1.conv_r.weight |     44     |
|  conv1.conv_r.bias  |     4      |
| conv1.conv_i.weight |     44     |
|  conv1.conv_i.bias  |     4      |
| conv2.conv_r.weight |     96     |
|  conv2.conv_r.bias  |     8      |
| conv2.conv_i.weight |     96     |
|  conv2.conv_i.bias  |     8      |
| conv3.conv_r.weight |    384     |
|  conv3.conv_r.bias  |     16     |
| conv3.conv_i.weight |    384     |
|  conv3.conv_i.bias  |     16     |
| conv4.conv_r.weight |    1536    |
|  conv4.conv_r.bias  |     32     |
| conv4.conv_i.weight |    1536    |
|  conv4.conv_i.bias  |     32     |
| conv5.conv_r.weight |    3072    |
|  conv5.conv_r.bias  |     32     |
| conv5.conv_i.weight |    3072    |
|  conv5.conv_i.bias  |     32     |
|      fc.weight      |     64     |
|       fc.bias       |     1      |
+---------------------+------------+
T

10634

### CNN_FE

In [23]:
SAVED_IN = os.path.abspath(DIR + "/trained_models" + "/CNN_fe")
MODEL_NAME = "cnn_baseline.pth"
config_save_path = os.path.join(SAVED_IN, MODEL_NAME).replace("pth", "config")
with open(config_save_path, "rb") as f:
    _config = pickle.load(f)

# dot access
config = dotdict(_config)

cnn = CNN_baseline_fe(
    n_features=config.n_features,
    base_channels=config.base_channels,
    device=device,
    activation_fn=dict_act_complex[config.activation],
    use_fe=config.use_fe,
    use_dropout=config.use_dropout,
    use_bn=config.use_bn,
).to(device)


CNN_baseline_fe(
  (conv1): Conv1d(1, 8, kernel_size=(22,), stride=(11,))
  (conv2): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
  (conv3): Conv1d(16, 32, kernel_size=(3,), stride=(2,))
  (conv4): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (conv_block1): Sequential(
    (0): Conv1d(1, 8, kernel_size=(22,), stride=(11,))
    (1): ComplexSigmoid()
  )
  (conv_block2): Sequential(
    (0): Conv1d(8, 16, kernel_size=(3,), stride=(2,))
    (1): ComplexSigmoid()
  )
  (conv_block3): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(2,))
    (1): ComplexSigmoid()
  )
  (conv_block4): Sequential(
    (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
    (1): ComplexSigmoid()
  )
)

In [24]:
count_parameters_table(cnn)

+--------------+------------+
|   Modules    | Parameters |
+--------------+------------+
|    weight    |    242     |
| conv1.weight |    176     |
|  conv1.bias  |     8      |
| conv2.weight |    384     |
|  conv2.bias  |     16     |
| conv3.weight |    1536    |
|  conv3.bias  |     32     |
| conv4.weight |    6144    |
|  conv4.bias  |     64     |
|  fc.weight   |    128     |
|   fc.bias    |     1      |
+--------------+------------+
Total Trainable Params: 8731


8731